# Experimental FoM values are calculated

According to the method layed out in [U. Forsberg et al., Physics Letters B](http://www.sciencedirect.com/science/article/pii/S0375947416300768?via%3Dihub).

In [1]:
import numpy as np
import os, sys, yaml, re
from IPython.display import display, Markdown, Latex #Can write latex too!!!!

In [2]:
class DecayChain(yaml.YAMLObject):
    
    yaml_tag = u'!DecayChain'
    
    def __init__(self, idd="", be="", pi="", ie="", it="", ea=[], eas=[], ta=[]):
        self.ID = idd
        self.BeamEnergy = be
        self.Pixel = pi
        self.ImplantEnergy = ie
        self.ImplantTime = it
        self.EAlpha, self.EAlphaSigma, self.TAlpha = ea, eas, ta

class SetDecayChains:
    
    def __init__(self, path='', ids=[]):
        s_files = " ".join(os.listdir(path))
        files = []
        for s in ids:
            files += (sorted(re.findall(string=s_files, pattern="Chain"+s+"\d+.yml")))
        print("Reading the following files:", files)
        self.Chains = []
        for f in files:
            f_in = open(path+f, 'r')
            self.Chains.append(yaml.load(f_in))
            f_in.close()

In [3]:
s_path = "E115_Chains/"
s_id = ["14", "11", "17"]

setDC = SetDecayChains(s_path, s_id)
max_steps = 0
for chain in setDC.Chains:
    if len(chain.TAlpha) > max_steps:
        max_steps = len(chain.TAlpha)
print("Max steps=", max_steps)
times = np.zeros((len(setDC.Chains), max_steps))
for row, chain in enumerate(setDC.Chains):
    steps = len(chain.TAlpha)
    for i in range(max_steps):
        if i < steps:
            times[row][i] = chain.TAlpha[i]
        else:
            times[row][i] = np.nan       
times

Reading the following files: ['Chain1401.yml', 'Chain1402.yml', 'Chain1403.yml', 'Chain1404.yml', 'Chain1405.yml', 'Chain1406.yml', 'Chain1407.yml', 'Chain1101.yml', 'Chain1102.yml', 'Chain1103.yml', 'Chain1104.yml', 'Chain1701.yml', 'Chain1702.yml', 'Chain1703.yml']
Max steps= 3


array([[  2.27000000e-01,   3.78000000e-01,              nan],
       [  6.45000000e-02,   3.66000000e-01,              nan],
       [  2.61000000e-01,   1.15000000e+00,   3.43000000e-01],
       [  1.46000000e+00,   2.62000000e-02,   4.32000000e-01],
       [  3.45000000e-01,   3.69000000e-01,   1.44000000e+01],
       [  2.10000000e-01,   1.05000000e+00,   8.27000000e+00],
       [  8.15000000e-01,   2.33000000e+00,   2.89000000e+00],
       [  2.56200000e-01,   1.40270000e+00,   1.97750000e+00],
       [  6.61000000e-02,   1.55000000e+00,   2.36380000e+00],
       [  2.35070000e+00,   2.25822000e+01,   6.01855000e+01],
       [  5.36000000e-02,   4.67100000e-01,   9.08000000e-02],
       [  2.14000000e-01,   1.54000000e+00,   7.57000000e+00],
       [  5.91000000e-02,   8.24000000e-01,              nan],
       [  4.55000000e-02,   1.42000000e-02,              nan]])

In [4]:
def fom_smeared(N_j, t_bar, times):
    ret = np.zeros((len(times[:]), len(N_j)))
    for i, t in enumerate(times[:]):
        temp = np.empty(len(t))
        prod = np.empty(len(t))
        np.multiply(N_j, t_bar, out=prod)
        np.power(prod, np.subtract(N_j,1), out=temp) #numerator
        np.divide(temp, np.power(np.add(prod, t), N_j), out=temp) #division with denominator
        np.multiply(temp, np.multiply(t, np.subtract(N_j, 1)), out=temp)
        ret[i] = temp
    return ret

In [5]:
N_j = np.count_nonzero(~np.isnan(times), axis=0)
N_j

array([14, 14, 10])

In [6]:
t_bar = np.nanmean(times, axis=0)
t_bar

array([ 0.45912143,  2.4321    ,  9.85226   ])

## Table 4 values and here individual

In [7]:
fom = fom_smeared(N_j, t_bar, times)
fom

array([[ 0.28241857,  0.12365139,         nan],
       [ 0.11343299,  0.12031173,         nan],
       [ 0.30235879,  0.27578495,  0.03026268],
       [ 0.16815648,  0.00989597,  0.03777367],
       [ 0.3355981 ,  0.12114996,  0.33621655],
       [ 0.27079382,  0.26203541,  0.33741097],
       [ 0.30989659,  0.35216391,  0.19771698],
       [ 0.29979615,  0.30432922,  0.14808697],
       [ 0.11584649,  0.31732229,  0.17035299],
       [ 0.06054081,  0.00695519,  0.04672682],
       [ 0.09650753,  0.1473675 ,  0.00821849],
       [ 0.27363418,  0.31651753,  0.32984525],
       [ 0.10515417,  0.22509897,         nan],
       [ 0.08337026,  0.00538998,         nan]])

In [8]:
fom[:, 0]

array([ 0.28241857,  0.11343299,  0.30235879,  0.16815648,  0.3355981 ,
        0.27079382,  0.30989659,  0.29979615,  0.11584649,  0.06054081,
        0.09650753,  0.27363418,  0.10515417,  0.08337026])

### Geometric and final average

In [11]:
def g_nan_mean(data):
    ret = np.empty(np.shape(data)[0])
    for i in range(len(data[:, 0])):
        temp = 1
        steps = 0
        for j in range(np.shape(data)[1]):
            if not np.isnan(data[i,j]):
                temp *= data[i,j]
            else:
                break
            steps += 1
        ret[i] = temp**(1./steps)
    return ret
    
#    if axis == -1:
#        ret = data.flatten()
#    else:
#        ret = data[axis]
#    print(ret)

fom_geom = g_nan_mean(fom)
fom_geom

array([ 0.18687282,  0.11682174,  0.13614452,  0.03976068,  0.23910424,
        0.28821681,  0.27839955,  0.23817464,  0.18432224,  0.02699652,
        0.04889355,  0.30569813,  0.15385089,  0.02119821])

In [12]:
fom_final = np.mean(fom_geom)
fom_final

0.16174675263817667